# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118355e+02     1.514427e+00
 * time: 0.8001198768615723
     1     9.940476e+00     8.086351e-01
 * time: 2.2900118827819824
     2    -1.178393e+01     9.592551e-01
 * time: 2.9428298473358154
     3    -3.417813e+01     7.180333e-01
 * time: 3.9324448108673096
     4    -4.740685e+01     5.244211e-01
 * time: 4.884706974029541
     5    -5.697417e+01     2.064539e-01
 * time: 5.860307931900024
     6    -5.987293e+01     1.011426e-01
 * time: 6.52989387512207
     7    -6.097896e+01     4.854106e-02
 * time: 7.18466591835022
     8    -6.142180e+01     4.399788e-02
 * time: 7.821630954742432
     9    -6.167193e+01     3.210368e-02
 * time: 8.488303899765015
    10    -6.187282e+01     2.800966e-02
 * time: 9.127388000488281
    11    -6.200954e+01     2.020580e-02
 * time: 9.766714811325073
    12    -6.208004e+01     1.792141e-02
 * time: 10.419861793518066
    13    -6.212429e+01     1.433366e-02
 * time: 11.074052810668945
   

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671067
    AtomicLocal         -18.8557667
    AtomicNonlocal      14.8522636
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336821

    total               -62.261666460406